In [3]:
import sparknlp  # χρειάζεται μετά στον bert , επειδή χρησιμοποιεί πακέτα από spark nlp

sparknlp.start(m1=True)   # m1=true ειδικη περιπτωση για διαθεση m1 processor, αν θελουμε χρηση gpu πχ gpu=True

import findspark          # initializing findspark (not neccessary)
findspark.init()

from pyspark import SparkContext       # create spark context
sc = SparkContext.getOrCreate()

import pyspark
from pyspark.sql import SparkSession             # create spark session
spark = SparkSession.builder \
    .appName("SparkProjectBobotasMakris.com") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.0.0").getOrCreate()          
    

    
from pyspark.sql.types import *
from pyspark.sql import functions as F
#from sparknlp.annotator import *   
#from sparknlp.base import *                               # those in comments needed later on sparknlp using processes like bert or word embeddings.
#from sparknlp.annotator import *
#from pyspark.ml import Pipeline
#from sparknlp import *
#from sparknlp.common import *
#from sparknlp.training import *
df = spark.read.option("header","true") \
  .option("sep", ",") \
  .option("multiLine", "true") \
  .option("quote","\"") \
  .option("escape","\"")\
  .option("ignoreTrailingWhiteSpace", True) \
  .csv("/Users/evangelosgeraridis/Desktop/Big Data Management and Mining Methods/job_postings.csv")


# ο πάνω τρόπος ανάγνωσης βοηθά, καθώς αν βάλουμε μόνο sep με κόμμα μπερδεύει τα στοιχεία (π.χ. στη 17η γραμμή του fraudulent αντί για 0 ή 1 διάβαζε 
# κάτι από τη δίπλα στήλη string (αν βάζαμε μόνο option ( sep , ,)


df = df.drop(*[c for c in df.columns if c not in {"description","fraudulent"}])       # κρατάμε μόνο τις 2 στήλες που θέλουμε 
df.printSchema()

df.show(30)



df.select([F.count(F.when(F.isnan(c), c)).alias(c) for c in df.columns]).show() # θέλω απλά να δω αν υπάρχουν nan values στο  df που 
# έχει στήλες μόνο την job description και τη fraudulent( το new_df έχει και array των words της στήλης description)
#,συνεπώς κοιτάμε απλά τις 2 στήλες ενδιαφέροντος. Τα έβαλα να εμφανίζονται σε μια γραμμή , 
# βλέπουμε ότι δεν υπάρχουν συνεπώς δε χρειαζόμαστε κάποια προεπεξεργασία (μέτρηση nan βγάζει 0 τω πλήθος) 



from pyspark.ml.feature import StopWordsRemover
   
new_df = df.withColumn('words',F.split(F.col('description'),' ')) # παιρνουμε χωρια καθε λεξη

new_df.show(10) # από ότι έψαξα τα μοντάλα με τα οποία θέλουμε να ασχοληθου΄με εδώ δεν είναι case sesitive (δε υπάρχει διαφορά μεταξύ κεφαλαίων
# ή μικρών γραμμάτων) οπότε είπα να χρησιμοποιήσω κάτι απλό και όχι κάποιο tokenizer
stop_words = StopWordsRemover(inputCol = 'words' , outputCol = 'filtered')   # λεξεις με stowordsrmover γινονται εν τελει filtered(no stopwords)
StopWordsRemover(inputCol="words", outputCol="filtered") 
new_df = stop_words.transform(new_df)
new_df.show(50)
new_df.printSchema()

# αφου το new_df εχει ιδιες στηλες με το df & τη words + filtered βλέπουμε λίγο τα στατιστικά αυτού του dataset
# για τη στήλη fraudulent ποσα μηδενικα εχει ως εγγραφες κυριως -> dataset balance

new_df.describe(['fraudulent']).show() # θέλουμε να δούμε αρχικά πόσες εγγραφές έχει το fraudulent , βλέπουμε ότι έχει 17880
new_df.filter(F.col("fraudulent")==0).count() # για να δούμε κατά πόσο υπάρχει ανισσόροπία στο df μετράμε τα μηδενικά 
# βλέπουμε ότι λίγο παραπάνω από 95% των fraudulent εγγραφών είναι 0 -> highly inbalanced dataset
# μια στρατηγική θα είναι μετά για τα training sets να πάρουμε δημιουργήσουμε νέες εγγραφές  fake 1 




root
 |-- description: string (nullable = true)
 |-- fraudulent: string (nullable = true)



+--------------------+----------+
|         description|fraudulent|
+--------------------+----------+
|Food52, a fast-gr...|         0|
|Organised - Focus...|         0|
|Our client, locat...|         0|
|THE COMPANY: ESRI...|         0|
|JOB TITLE: Itemiz...|         0|
|Job OverviewApex ...|         0|
|Your Responsibili...|         0|
|Who is Airenvy?He...|         0|
|Implementation/Co...|         0|
|The Customer Serv...|         0|
|Position : #URL_8...|         0|
|TransferWise is t...|         0|
|The Applications ...|         0|
|Event Industry In...|         0|
|Are you intereste...|         0|
|About Vault Drago...|         0|
|We are looking fo...|         0|
|Government fundin...|         0|
|Kettle is hiring ...|         0|
|Experienced Proce...|         0|
|IntelliBright is ...|         0|
|Want to be part o...|         0|
|The position repo...|         0|
|#URL_eda2500ddced...|         0|
|We are a canary w...|         0|
|Hello,Wish you ar...|         0|
|We are curren

+-----------+----------+
|description|fraudulent|
+-----------+----------+
|          0|         0|
+-----------+----------+

+--------------------+----------+--------------------+
|         description|fraudulent|               words|
+--------------------+----------+--------------------+
|Food52, a fast-gr...|         0|[Food52,, a, fast...|
|Organised - Focus...|         0|[Organised, -, Fo...|
|Our client, locat...|         0|[Our, client,, lo...|
|THE COMPANY: ESRI...|         0|[THE, COMPANY:, E...|
|JOB TITLE: Itemiz...|         0|[JOB, TITLE:, Ite...|
|Job OverviewApex ...|         0|[Job, OverviewApe...|
|Your Responsibili...|         0|[Your, Responsibi...|
|Who is Airenvy?He...|         0|[Who, is, Airenvy...|
|Implementation/Co...|         0|[Implementation/C...|
|The Customer Serv...|         0|[The, Customer, S...|
+--------------------+----------+--------------------+
only showing top 10 rows



22/07/01 15:15:24 WARN StopWordsRemover: Default locale set was [en_GR]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
22/07/01 15:15:24 WARN StopWordsRemover: Default locale set was [en_GR]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


+--------------------+----------+--------------------+--------------------+
|         description|fraudulent|               words|            filtered|
+--------------------+----------+--------------------+--------------------+
|Food52, a fast-gr...|         0|[Food52,, a, fast...|[Food52,, fast-gr...|
|Organised - Focus...|         0|[Organised, -, Fo...|[Organised, -, Fo...|
|Our client, locat...|         0|[Our, client,, lo...|[client,, located...|
|THE COMPANY: ESRI...|         0|[THE, COMPANY:, E...|[COMPANY:, ESRI, ...|
|JOB TITLE: Itemiz...|         0|[JOB, TITLE:, Ite...|[JOB, TITLE:, Ite...|
|Job OverviewApex ...|         0|[Job, OverviewApe...|[Job, OverviewApe...|
|Your Responsibili...|         0|[Your, Responsibi...|[Responsibilities...|
|Who is Airenvy?He...|         0|[Who, is, Airenvy...|[Airenvy?Hey, the...|
|Implementation/Co...|         0|[Implementation/C...|[Implementation/C...|
|The Customer Serv...|         0|[The, Customer, S...|[Customer, Servic...|
|Position : 

+-------+-------------------+
|summary|         fraudulent|
+-------+-------------------+
|  count|              17880|
|   mean|0.04843400447427293|
| stddev|0.21468751586002602|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



17014

In [ ]:
#WORD 2 VEC

from pyspark.ml.feature import Word2Vec
word2Vec = Word2Vec(vectorSize=7, minCount=0, seed = 21, inputCol="filtered", outputCol="word_vectors")
model = word2Vec.fit(new_df)
# κάνουμε κάθε λέξη (απο τη στηλη χωρις stopwprds) -> διανυσμα διαστασης 7 ,seed αρχικοποιηση ουσιαστικα 
# mincount Ignores all words with total frequency lower than this. lower than zero-> δεν εμφανιζεται καν στο dataset
word2vec_df= model.transform(new_df)
word2vec_df = word2vec_df.drop(*[c for c in word2vec_df.columns if c not in {"word_vectors","fraudulent"}])

word2vec_df.show()